## Condensed into functions
* Feed in input when initializing class and then calculate all the basic stuff that many functions need fe nr_zero_discharge_layers

## Todo
* Move to JulesD3D
* Should be able to handle multiple discharge start times
* Should make use of generic write bc functions bcc.py
* Should be able to declare volume percentages of sediments and then use function to translate this into a kg/m3
* Should be able to handle more than two sediments
* Read grid depth at bc, declare flow height then calculate nr of gridlayers needed to reach that height

In [1]:
import os
import numpy as np
from JulesD3D.utils import formatSci
from JulesD3D.mdf import read
from IPython.display import Markdown as md
import pandas as pd
from decimal import Decimal

# User defined input

In [2]:
nr_discharge_layers = 53
init_depth = 300   # [m] at bc location # ureg.meter 
density_sed = 2650 # [kg/m3]

cell_width = 200   # at bc location # ureg.meter [m] 
cell_length = 200  # at bc location # ureg.meter [m]

density_sed = 2650 # [kg/m3] # should be read from sed file ideally
discharge_total = 4500 # [m3/s]

sand_mass_conc = 19.75
silt_mass_conc = 19.75 

sediments = [{'name': 'sand', "mass_conc": sand_mass_conc}, {'name': 'silt', "mass_conc": silt_mass_conc}]

start_discharge_time = 9 # [min]
discharge_duration = 39 # [min]

## Read some things from mdf file

In [3]:
mdf_filename = '/Users/julesblom/Dropbox/TU/Master/Thesis/Runs/Current/ResetEvery5_Slope0.85/Run01/36km_200m_W60Channel.mdf'
mdf_dict, _ = read(mdf_filename)

In [4]:
sigma_layer_percentages = mdf_dict['Thick']
nr_sigma_layers = int(mdf_dict['MNKmax'][2])
timestep = mdf_dict['Dt'][0] # [min]
end_time = mdf_dict['Tstop'][0] # [min]

## Calculate some more basics

In [5]:
nr_of_timesteps = int(end_time / timestep)
nr_zero_discharge_layers = int(nr_sigma_layers - nr_discharge_layers)
end_discharge_time = start_discharge_time + discharge_duration # [min]

In [6]:
input_dict = dict(timestep = timestep, nr_sigma_layers=nr_sigma_layers, start_discharge_time=start_discharge_time,
                  discharge_duration=discharge_duration, nr_discharge_layers=nr_discharge_layers)

# All these functions

In [7]:
def allZeroesRecords(nr_sigma_layers=80):
    all_zeros_line = '  '.join('0.0000000e+000' for i in range(nr_sigma_layers))
    line_all_zero = f"{all_zeros_line}  {all_zeros_line}" # left + right
    
    return line_all_zero

In [8]:
def makeBcTimes(timestep=0.3, start_discharge_time=15, discharge_duration=45):
    # Discharge is linearly interpolated between timesteps
    # Therefore, we need to add one timestep before the start and one timestep after in which discharges are zero
    end_discharge_time = start_discharge_time + discharge_duration # [min]
    pre_start_discharge_time = start_discharge_time - timestep  # [min]
    post_end_discharge_time = end_discharge_time + timestep  # [min]    

    times = [formatSci(0), 
             formatSci(pre_start_discharge_time), 
             formatSci(start_discharge_time),
             formatSci(end_discharge_time), 
             formatSci(post_end_discharge_time), 
             formatSci(end_time)
            ]
    
    return times

In [18]:
def calcDischargePerCell(discharge_total=4500, sigma_layer_percentages=[], nr_zero_discharge_layers=0):
    discharge_layer_sigma_percentages = sigma_layer_percentages[nr_zero_discharge_layers:] # select only the discharge parts
    discharge_layers_sigma_percentage_sum = np.array(discharge_layer_sigma_percentages).sum() # sum it

    print(f"Now divide {discharge_total} $m^3/s$ discharge among the bottom {discharge_layers_sigma_percentage_sum}% height of sigma layers")
    discharge_records_list = discharge_layer_sigma_percentages/discharge_layers_sigma_percentage_sum * discharge_total
    
    return discharge_records_list

In [19]:
def makeDischargeRecords(discharge_bc_list=None, nr_sigma_layers=80, nr_discharge_layers = 53):
    '''
    Pass a list of float values for discharge layer 
    Returns a formatted string of records
    '''
    if not len(discharge_bc_list):
        raise Exception("discharge_records should be be a list")
    
    nr_zero_discharge_layers = nr_sigma_layers - nr_discharge_layers  # move this to class self props

    discharge_records = '  '.join(formatSci(discharge) for discharge in discharge_bc_list)
    
    print(f"The first {nr_zero_discharge_layers} 'cells' are all zeroes, then the next {nr_discharge_layers} layers contain the discharge values")
    
    zero_records_discharge_layer = '  '.join('0.0000000e+000' for i in range(nr_zero_discharge_layers))
    # first the cells where discharge is zero, then the cells that have values for discharge 
    discharge_line = f"{zero_records_discharge_layer}  {discharge_records}"

    # Each line has two discharge 'points' so repeat it twice
    complete_line_records = f'{discharge_line}  {discharge_line}' # A + B

    return complete_line_records

In [20]:
line_all_zero = allZeroesRecords(nr_sigma_layers)

In [21]:
# times = makeBcTimes(timestep, start_discharge_time, discharge_duration)

In [22]:
def generateVerticalBC(filename='untitled', timestep = 0.3, nr_sigma_layers=80, start_discharge_time=0,
                       discharge_duration=0, nr_discharge_layers=53):
    if discharge_duration == 0:
        raise Exception("discharge_duration cannot be 0")
    
    if not Decimal(discharge_duration) % Decimal(timestep) < 10e-9:
        raise Exception("Discharge duration is not a multiple of timestep")

    times = makeBcTimes(timestep, start_discharge_time, discharge_duration)    
    line_all_zero = allZeroesRecords(nr_sigma_layers)
        
    # --- Write Concentration file (.bcc) ---
    # time + A + B
    bcc_filename = filename + ".bccrecords"
    with open(bcc_filename, 'w') as f:
        # why not use a f multiline string here f'''{ line below} '''
        # what if more than 6 records? this approach is inflexible
        # for each discharge write these six lines
            # for time in times:
            
        for sed in sediments:
            f.write(sed['name'] + '\n')
            conc_discharge_list = [sed['mass_conc'] for i in range(nr_discharge_layers)]

            line_with_conc_records = makeDischargeRecords(conc_discharge_list)

            for i, time in enumerate(times):
                if i == 2 or i == 3:
                    write_line =  f" {time}  {line_with_conc_records}\n"
                    f.write(write_line)
                else:
                    write_line = f" {time}  {line_all_zero}\n"
                    f.write(write_line)                    
            
        # one discharge always follows this sequence of 6 time values with discharge records at the 3rd and 4th place
#         linesToWrite = [
#             "sand\n",
#             f" {times[0]}  {line_all_zero}\n",
#             f" {times[1]}  {line_all_zero}\n",
#             f" {times[2]}  {line_with_sand_conc_records}\n",
#             f" {times[3]}  {line_with_sand_conc_records}\n",
#             f" {times[4]}  {line_all_zero}\n",
#             f" {times[5]}  {line_all_zero}\n",
#             "\n",
#             "silt\n"
#             f" {times[0]}  {line_all_zero}\n",
#             f" {times[1]}  {line_all_zero}\n",
#             f" {times[2]}  {line_with_silt_conc_records}\n",
#             f" {times[3]}  {line_with_silt_conc_records}\n",
#             f" {times[4]}  {line_all_zero}\n",
#             f" {times[5]}  {line_all_zero}\n"
#         ]
#         f.writelines(line for line in linesToWrite)
        
    # discharges inflow of water (different for gridcells)
    nr_zero_discharge_layer = nr_sigma_layers - nr_discharge_layers
    discharges_list = calcDischargePerCell(discharge_total, sigma_layer_percentages, nr_zero_discharge_layer)
    line_with_discharge_records = makeDischargeRecords(discharges_list, nr_sigma_layers, nr_discharge_layers)        
    # Format: time + A + B
    bct_filename = filename + ".bctrecords"
    with open(bct_filename, 'w') as bct_file:
        linesToWrite = [
            f" {times[0]}  {line_all_zero}\n",
            f" {times[1]}  {line_all_zero}\n",
            f" {times[2]}  {line_with_discharge_records}\n",
            f" {times[3]}  {line_with_discharge_records}\n",
            f" {times[4]}  {line_all_zero}\n",
            f" {times[5]}  {line_all_zero}\n"
        ]
        bct_file.writelines(line for line in linesToWrite) 

# And now, magic!

In [23]:
generateVerticalBC('test', **input_dict)

The first 27 'cells' are all zeroes, then the next 53 layers contain the discharge values
The first 27 'cells' are all zeroes, then the next 53 layers contain the discharge values
Now divide 4500 $m^3/s$ discharge among the bottom 16.42% height of sigma layers
The first 27 'cells' are all zeroes, then the next 53 layers contain the discharge values
